# MCP Gemini Client - Interactive Breakdown

This notebook breaks down the `client-gemini.py` into interactive components for step-by-step execution.

## Step 1: Import Required Libraries

In [1]:
import asyncio
import os
from contextlib import AsyncExitStack
from typing import Any, List, Optional

import nest_asyncio
from dotenv import load_dotenv
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from google import genai
from google.genai import types

# Allow nested event loops (required for Jupyter)
nest_asyncio.apply()

print("✅ All imports successful!")

✅ All imports successful!


## Step 2: Load Environment Variables

In [2]:
# Load environment variables from .env file
load_dotenv("../.env")

# Get API key and model from environment
api_key = os.getenv("GEMINI_API_KEY")
model = os.getenv("GEMINI_MODEL", "gemini-2.5-flash")

if not api_key:
    raise ValueError("❌ GEMINI_API_KEY is not set in .env file")

print(f"✅ API Key loaded: {api_key[:20]}...")
print(f"✅ Model: {model}")

✅ API Key loaded: AIzaSyAkFnPWLleFVjIP...
✅ Model: gemini-2.5-flash


## Step 3: Initialize Gemini Client

In [3]:
# Create Gemini client
gemini_client = genai.Client(api_key=api_key)

print("✅ Gemini client initialized")
print(f"   Using model: {model}")

✅ Gemini client initialized
   Using model: gemini-2.5-flash


## Step 4: Connect to MCP Server

This connects to the MCP server running `server.py` via stdio transport.

In [4]:
# Initialize session variables
session: Optional[ClientSession] = None
exit_stack = AsyncExitStack()
stdio: Optional[Any] = None
write: Optional[Any] = None

async def connect_to_server(server_script_path: str = "server.py"):
    global session, stdio, write, exit_stack
    
    # Configure server parameters
    server_params = StdioServerParameters(
        command="python",
        args=[server_script_path],
    )
    
    # Connect to server via stdio
    stdio_transport = await exit_stack.enter_async_context(
        stdio_client(server_params)
    )
    stdio, write = stdio_transport
    
    # Create and initialize session
    session = await exit_stack.enter_async_context(
        ClientSession(stdio, write)
    )
    
    await session.initialize()
    
    # List available tools
    tools_result = await session.list_tools()
    print("\n✅ Connected to MCP server with tools:")
    for tool in tools_result.tools:
        print(f"  - {tool.name}: {tool.description[:80]}...")
    
    return tools_result

# Run the connection
tools_result = await connect_to_server("server.py")


✅ Connected to MCP server with tools:
  - get_knowledge_base: Retrieve the knowledge base containing sarcastic/funny responses to common workp...


## Step 5: Convert MCP Tools to Gemini Format

Convert the MCP tools into Gemini's function declaration format.

In [5]:
async def get_gemini_tools() -> List[types.Tool]:
    """Convert MCP tools to Gemini format"""
    tools_result = await session.list_tools()
    
    function_decls = [
        types.FunctionDeclaration(
            name=tool.name,
            description=tool.description,
            parameters=tool.inputSchema,
        )
        for tool in tools_result.tools
    ]
    
    return [types.Tool(function_declarations=function_decls)]

# Get tools in Gemini format
gemini_tools = await get_gemini_tools()
print(f"✅ Converted {len(gemini_tools[0].function_declarations)} tools to Gemini format")
for func_decl in gemini_tools[0].function_declarations:
    print(f"  - {func_decl.name}")

✅ Converted 1 tools to Gemini format
  - get_knowledge_base


## Step 6: Send Query to Gemini (First Call)

Send a query to Gemini with the available tools. Gemini will decide whether to call the tool.

In [6]:
# Define the query
query = "Hey, how are you doing?"
print(f"\n📤 Query: {query}\n")

# Define system instruction
system_instruction = (
    "You are a workplace assistant with access to a knowledge base of sarcastic/funny responses. "
    "When the user asks a question, ALWAYS check the knowledge base first using the available tool "
    "to see if there's a matching response. If found, use that response."
)

# Call Gemini with tools
response = gemini_client.models.generate_content(
    model=model,
    contents=query,
    config=types.GenerateContentConfig(
        tools=gemini_tools,
        system_instruction=system_instruction,
    ),
)

print("✅ Received response from Gemini")
print(f"\nResponse parts: {len(response.candidates[0].content.parts)}")


📤 Query: Hey, how are you doing?

✅ Received response from Gemini

Response parts: 1


In [7]:
response

GenerateContentResponse(
  candidates=[
    Candidate(
      content=Content(
        parts=[
          Part(
            function_call=FunctionCall(
              args=<... Max depth ...>,
              name=<... Max depth ...>
            ),
            thought_signature=b'\n\x9c\x02\x01r\xc8\xda|\xd8\x85~\t\x8a\x18\xbd:\xea\xac\xe3\xd2\x05q\xb9\xe8\x06\xbf:DCyIb\xb5\xa3\x05\x16)\xd4\xb4U\xc9a\n[\xaeY\xdb\x1e\x83\x90\x1e#\xa5\x85\x80N\x00\x94\xb35\xd4\xa7q\xbd\x060\x0b,\xd3\x8f\x08>0\x8b\xdb\x10\xda\x85\xabE\x81SiMA>\x96#Gh m\x80\xca\xd7\xde\xf5...'
          ),
        ],
        role='model'
      ),
      finish_reason=<FinishReason.STOP: 'STOP'>,
      index=0
    ),
  ],
  model_version='gemini-2.5-flash',
  response_id='1nldafKSD_6_juMPxuza8Qk',
  sdk_http_response=HttpResponse(
    headers=<dict len=11>
  ),
  usage_metadata=GenerateContentResponseUsageMetadata(
    candidates_token_count=12,
    prompt_token_count=147,
    prompt_tokens_details=[
      ModalityTokenCount(
   

## Step 7: Check if Tool Was Called

Inspect the response to see if Gemini decided to call any tools.

In [10]:
parts = response.candidates[0].content.parts
print(parts)

[Part(
  function_call=FunctionCall(
    args={},
    name='get_knowledge_base'
  ),
  thought_signature=b'\n\x9c\x02\x01r\xc8\xda|\xd8\x85~\t\x8a\x18\xbd:\xea\xac\xe3\xd2\x05q\xb9\xe8\x06\xbf:DCyIb\xb5\xa3\x05\x16)\xd4\xb4U\xc9a\n[\xaeY\xdb\x1e\x83\x90\x1e#\xa5\x85\x80N\x00\x94\xb35\xd4\xa7q\xbd\x060\x0b,\xd3\x8f\x08>0\x8b\xdb\x10\xda\x85\xabE\x81SiMA>\x96#Gh m\x80\xca\xd7\xde\xf5...'
)]


In [8]:
parts = response.candidates[0].content.parts
tool_called = False

for i, part in enumerate(parts):
    print(f"\nPart {i+1}:")
    if part.function_call:
        tool_called = True
        print(f"  ✅ Function Call Detected!")
        print(f"  Function name: {part.function_call.name}")
        print(f"  Arguments: {dict(part.function_call.args)}")
    elif part.text:
        print(f"  Text response: {part.text}")

if tool_called:
    print("\n✅ Gemini decided to call a tool!")
else:
    print("\n❌ No tool was called. Gemini responded directly.")


Part 1:
  ✅ Function Call Detected!
  Function name: get_knowledge_base
  Arguments: {}

✅ Gemini decided to call a tool!


## Step 8: Execute Tool Call via MCP

If a tool was called, execute it through the MCP server.

In [11]:
tool_parts = []

if tool_called:
    for part in parts:
        if part.function_call:
            fn_name = part.function_call.name
            args = dict(part.function_call.args)
            
            print(f"🔧 Executing tool: {fn_name}")
            print(f"   Arguments: {args}")
            
            # Call the MCP tool
            result = await session.call_tool(fn_name, arguments=args)
            
            print(f"\n✅ Tool result received:")
            print(f"   {result.content[0].text[:200]}...\n")
            
            # Prepare tool response for Gemini
            tool_parts.append(
                types.Part.from_function_response(
                    name=fn_name,
                    response={"result": result.content[0].text},
                )
            )
else:
    print("No tools to execute.")

🔧 Executing tool: get_knowledge_base
   Arguments: {}

✅ Tool result received:
   Here is the retrieved knowledge base:

Q1: Hey, how are you doing?
A1: 👍

Q2: Do you have a minute? I have some urgent work.
A2: ummmmmmmmmmm no

Q3: How is this session going?
A3: thinking to resign
...



## Step 9: Send Tool Result Back to Gemini (Follow-up Call)

Send the tool results back to Gemini to get the final response.

In [ ]:
if tool_called:
    print("📤 Sending tool results back to Gemini...\n")
    
    # Send follow-up with tool results
    followup = gemini_client.models.generate_content(
        model=model,
        contents=[
            query,
            response.candidates[0].content,
            types.Content(role="tool", parts=tool_parts),
        ],
        config=types.GenerateContentConfig(
            system_instruction=system_instruction,
        ),
    )
    
    final_response = followup.text
else:
    final_response = response.text

print("\n" + "="*60)
print("📥 FINAL RESPONSE:")
print("="*60)
print(f"\n{final_response}\n")
print("="*60)

## Step 10: Test with Different Queries

Try different queries from the knowledge base.

In [ ]:
async def process_query(query: str) -> str:
    """Process a query end-to-end"""
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print(f"{'='*60}\n")
    
    # Get tools
    tools = await get_gemini_tools()
    
    # First call to Gemini
    response = gemini_client.models.generate_content(
        model=model,
        contents=query,
        config=types.GenerateContentConfig(
            tools=tools,
            system_instruction=system_instruction,
        ),
    )
    
    # Check for tool calls
    parts = response.candidates[0].content.parts
    tool_parts = []
    tool_called = False
    
    for part in parts:
        if part.function_call:
            tool_called = True
            fn_name = part.function_call.name
            args = dict(part.function_call.args)
            
            # Execute tool
            result = await session.call_tool(fn_name, arguments=args)
            
            tool_parts.append(
                types.Part.from_function_response(
                    name=fn_name,
                    response={"result": result.content[0].text},
                )
            )
    
    # Get final response
    if not tool_called:
        return response.text
    
    followup = gemini_client.models.generate_content(
        model=model,
        contents=[
            query,
            response.candidates[0].content,
            types.Content(role="tool", parts=tool_parts),
        ],
        config=types.GenerateContentConfig(
            system_instruction=system_instruction,
        ),
    )
    
    return followup.text

# Test different queries
test_queries = [
    "Any updates?",
    "Do you have a minute?",
    "How is this session going?",
    "Why are you so quiet?"
]

for test_query in test_queries:
    result = await process_query(test_query)
    print(f"Response: {result}\n")

## Step 11: Cleanup

Close the MCP session and clean up resources.

In [ ]:
# Cleanup
await exit_stack.aclose()
print("✅ Cleaned up MCP session")